# Sparse HRT - GPU-Accelerated IICA Architecture (v0.5.0)

This notebook explores the **Sparse GPU Architecture** for HRT:

1. **Sparse Basics** - ImmutableSparseTensor, COO format on CUDA
2. **SparseAM** - Sparse Adjacency Matrix with context covariance
3. **SparseLattice** - Connection tracking for active indices only
4. **SparseHRT** - Complete sparse HRT with IICA properties
5. **Memory Comparison** - Dense vs Sparse (4GB vs ~2MB!)
6. **IICA Properties** - Immutability, Idempotence, Content Addressability
7. **Large Scale Test** - 100K+ edges on GPU

**Key Insight**: AM[i,j] = |row_context[i] ∩ col_context[j]| (context covariance, not TF)

---

## 1. Import Sparse GPU Architecture

In [1]:
import time
import warnings
import os

# Suppress warnings about incompatible secondary GPU (Quadro M1200)
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0")
warnings.filterwarnings("ignore", message=".*cuda capability.*")
warnings.filterwarnings("ignore", message=".*Quadro M1200.*")

import torch

# Sparse GPU Architecture (v0.5.0)
from core import (
    # Sparse Tensor Foundation
    ImmutableSparseTensor,
    get_device,
    sparse_zeros,
    sparse_from_dense,
    
    # Sparse HRT Components
    SparseHRT,
    SparseHRTConfig,
    SparseAM,
    SparseLattice,
    create_sparse_hrt,
    
    # Version
    __version__
)

print(f"Fractal Manifold Core v{__version__}")
print(f"Device: {get_device()}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

Fractal Manifold Core v0.5.1
Device: cuda
CUDA available: True
GPU: NVIDIA GeForce RTX 3060
VRAM: 11.7 GB


## 2. ImmutableSparseTensor - COO Format on CUDA

The foundation of sparse HRT is `ImmutableSparseTensor`:
- **COO format**: indices [2, nnz] + values [nnz]
- **Content-addressed**: name = SHA1 of sorted edges
- **Immutable**: all operations return new tensor
- **CUDA-accelerated**: lives on GPU

In [2]:
# Create empty sparse tensor (32K x 32K - same as dense HRT dimension)
device = str(get_device())
t1 = ImmutableSparseTensor.empty(32770, 32770, device)

print("=== Empty Sparse Tensor ===")
print(f"Shape: {t1.shape}")
print(f"NNZ: {t1.nnz}")
print(f"Device: {t1.device}")
print(f"Name: {t1.name[:32]}...")
print()

# Add edges (returns new tensor - immutable!)
t2 = t1.with_edge(100, 200, 5.0)
t3 = t2.with_edge(300, 400, 3.0)
t4 = t3.with_edge(100, 201, 7.0)

print("=== With 3 Edges ===")
print(f"NNZ: {t4.nnz}")
print(f"Edges: {t4.edges()}")
print(f"Name: {t4.name[:32]}...")
print()

# Query
print("=== Query ===")
print(f"get(100, 200): {t4.get(100, 200)}")
print(f"get(100, 201): {t4.get(100, 201)}")
print(f"get(999, 999): {t4.get(999, 999)} (not found)")
print(f"Active rows: {t4.active_rows()}")
print(f"Active cols: {t4.active_cols()}")

=== Empty Sparse Tensor ===
Shape: (32770, 32770)
NNZ: 0
Device: cuda:0
Name: fb8734a8192ec83207f097621362d85c...

=== With 3 Edges ===
NNZ: 3
Edges: [(100, 200, 5.0), (300, 400, 3.0), (100, 201, 7.0)]
Name: 68e4a0ffb376ab1b0f8309d7cd2b627b...

=== Query ===
get(100, 200): 5.0
get(100, 201): 7.0
get(999, 999): 0.0 (not found)
Active rows: {100, 300}
Active cols: {200, 201, 400}


## 3. Idempotence - Merge with Maximum

Sparse tensor merge uses element-wise maximum:
- `max(A, A) = A` (idempotent)
- `max(A, B) = max(B, A)` (commutative)
- Preserves IICA properties

In [3]:
# Create two overlapping sparse tensors
s1 = ImmutableSparseTensor.from_edges(1000, 1000, [
    (10, 20, 5.0),
    (30, 40, 3.0),
], device)

s2 = ImmutableSparseTensor.from_edges(1000, 1000, [
    (10, 20, 8.0),  # Same position, higher value
    (50, 60, 2.0),  # New edge
], device)

print("=== Before Merge ===")
print(f"s1: {s1.nnz} edges, name={s1.name[:16]}...")
print(f"s2: {s2.nnz} edges, name={s2.name[:16]}...")
print()

# Merge (element-wise maximum)
merged = s1.maximum(s2)
print("=== After Merge ===")
print(f"merged: {merged.nnz} edges")
print(f"get(10, 20): {merged.get(10, 20)} (max of 5.0 and 8.0)")
print(f"get(30, 40): {merged.get(30, 40)} (from s1)")
print(f"get(50, 60): {merged.get(50, 60)} (from s2)")
print()

# Self-merge idempotence
self_merged = s1.maximum(s1)
print("=== Idempotence ===")
print(f"s1.maximum(s1).name == s1.name: {self_merged.name == s1.name} ✓")

=== Before Merge ===
s1: 2 edges, name=bd580d21b16d3cf6...
s2: 2 edges, name=11bdf11a03d2c68d...

=== After Merge ===
merged: 3 edges
get(10, 20): 8.0 (max of 5.0 and 8.0)
get(30, 40): 3.0 (from s1)
get(50, 60): 2.0 (from s2)

=== Idempotence ===
s1.maximum(s1).name == s1.name: True ✓


## 4. SparseAM - Sparse Adjacency Matrix

`SparseAM` wraps `ImmutableSparseTensor` with HRT-specific operations:
- Tracks active rows/columns
- Supports intersection-based values (context covariance)
- Maintains IICA properties

In [4]:
# Create SparseAM
config = SparseHRTConfig(p_bits=10, h_bits=32)
print(f"=== SparseHRTConfig ===")
print(f"p_bits: {config.p_bits}")
print(f"h_bits: {config.h_bits}")
print(f"dimension: {config.dimension} (AM size)")
print(f"device: {config.device}")
print()

# Create empty AM
am = SparseAM.empty(config)
print(f"=== Empty SparseAM ===")
print(f"{am}")
print()

# Add edges
am1 = am.with_edge(100, 200, 5.0)
am2 = am1.with_edge(300, 400, 3.0)
am3 = am2.with_edge(100, 201, 7.0)

print(f"=== With 3 Edges ===")
print(f"{am3}")
print(f"Active rows: {am3.active_rows}")
print(f"Active cols: {am3.active_cols}")
print()

# Query neighbors
print(f"Row 100 neighbors: {am3.row_neighbors(100)}")
print(f"Col 200 neighbors: {am3.col_neighbors(200)}")

=== SparseHRTConfig ===
p_bits: 10
h_bits: 32
dimension: 32770 (AM size)
device: cuda

=== Empty SparseAM ===
SparseAM(nnz=0, active_rows=0, active_cols=0, device='cuda:0')

=== With 3 Edges ===
SparseAM(nnz=3, active_rows=2, active_cols=3, device='cuda:0')
Active rows: frozenset({100, 300})
Active cols: frozenset({200, 201, 400})

Row 100 neighbors: [(200, 5.0), (201, 7.0)]
Col 200 neighbors: [(100, 5.0)]


## 5. SparseLattice - Connection Tracking

`SparseLattice` tracks which indices are connected:
- `row_connections[i]` = set of columns connected to row i
- `col_connections[j]` = set of rows connected to column j

This enables computing **context covariance**: `|row_conns[i] ∩ col_conns[j]|`

In [5]:
# Build lattice from SparseAM
lattice = SparseLattice.from_sparse_am(am3)

print("=== SparseLattice ===")
print(f"{lattice}")
print()

print("=== Row Connections ===")
print(f"Row 100 connected to: {lattice.row_connections(100)}")
print(f"Row 100 cardinality: {lattice.row_cardinality(100)}")
print(f"Row 300 connected to: {lattice.row_connections(300)}")
print()

print("=== Column Connections ===")
print(f"Col 200 connected to: {lattice.col_connections(200)}")
print(f"Col 201 connected to: {lattice.col_connections(201)}")
print(f"Col 400 connected to: {lattice.col_connections(400)}")
print()

# Context covariance = intersection cardinality
# (In a real scenario, positions that share many connections are related)
print("=== Context Covariance ===")
print(f"|row_100 ∩ col_200|: {lattice.intersection_cardinality(100, 200)}")
print(f"|row_100 ∩ col_201|: {lattice.intersection_cardinality(100, 201)}")

=== SparseLattice ===
SparseLattice(rows=2, cols=3)

=== Row Connections ===
Row 100 connected to: frozenset({200, 201})
Row 100 cardinality: 2
Row 300 connected to: frozenset({400})

=== Column Connections ===
Col 200 connected to: frozenset({100})
Col 201 connected to: frozenset({100})
Col 400 connected to: frozenset({300})

=== Context Covariance ===
|row_100 ∩ col_200|: 0
|row_100 ∩ col_201|: 0


## 6. SparseHRT - Complete Sparse HRT

`SparseHRT` combines:
- `SparseAM`: Sparse adjacency matrix on GPU
- `SparseLattice`: Connection tracking
- `LUT`: Token disambiguation (FrozenSet of entries)
- `Config`: Parameters (p_bits, h_bits, tau, rho, epsilon)

All IICA properties are maintained.

In [6]:
# Create empty SparseHRT
hrt = create_sparse_hrt(p_bits=10, h_bits=32)

print("=== Empty SparseHRT ===")
print(f"{hrt}")
print(f"Name: {hrt.name[:32]}...")
print(f"Device: {hrt.device}")
print()

# Add edges (returns new HRT - immutable!)
hrt1 = hrt.with_edge(100, 200, 5.0)
hrt2 = hrt1.with_edge(300, 400, 3.0)
hrt3 = hrt2.with_edge(100, 201, 7.0)

print("=== With 3 Edges ===")
print(f"{hrt3}")
print(f"Active rows: {hrt3.am.active_rows}")
print(f"Active cols: {hrt3.am.active_cols}")
print(f"Lattice: {hrt3.lattice}")

=== Empty SparseHRT ===
SparseHRT(nnz=0, lut=0, step=0, mem=0.00MB, device='cuda:0')
Name: 1fb4ec7b258adc1f2cb9d6905f62f722...
Device: cuda:0

=== With 3 Edges ===
SparseHRT(nnz=3, lut=0, step=0, mem=0.00MB, device='cuda:0')
Active rows: frozenset({100, 300})
Active cols: frozenset({200, 201, 400})
Lattice: SparseLattice(rows=2, cols=3)


## 7. IICA Properties - Verification

Testing the three IICA properties:
1. **Immutability**: Original not modified
2. **Idempotence**: `merge(A, A).am == A.am`
3. **Content Addressability**: Same content → same hash

In [7]:
# 1. Immutability
original_name = hrt.name
hrt_modified = hrt.with_edge(999, 999, 1.0)
print("=== Immutability ===")
print(f"Original unchanged: {hrt.name == original_name} ✓")
print(f"New HRT created: {hrt_modified.name != original_name} ✓")
print()

# 2. Idempotence (at AM level - merge increments step for evolution semantics)
am_a = SparseAM.from_edges(config, [(10, 20, 5.0), (30, 40, 3.0)])
am_b = am_a.merge(am_a)
print("=== Idempotence (AM level) ===")
print(f"am_a.name: {am_a.name[:24]}...")
print(f"am_a.merge(am_a).name: {am_b.name[:24]}...")
print(f"Idempotent: {am_a.name == am_b.name} ✓")
print()

# 3. Content Addressability
hrt_x = create_sparse_hrt()
hrt_y = create_sparse_hrt()  # Same empty content
print("=== Content Addressability ===")
print(f"hrt_x.name: {hrt_x.name[:24]}...")
print(f"hrt_y.name: {hrt_y.name[:24]}...")
print(f"Same content = same hash: {hrt_x.name == hrt_y.name} ✓")

=== Immutability ===
Original unchanged: True ✓
New HRT created: True ✓

=== Idempotence (AM level) ===
am_a.name: 1cd49265f3771458fc7aaa55...
am_a.merge(am_a).name: 1cd49265f3771458fc7aaa55...
Idempotent: True ✓

=== Content Addressability ===
hrt_x.name: 1fb4ec7b258adc1f2cb9d690...
hrt_y.name: 1fb4ec7b258adc1f2cb9d690...
Same content = same hash: True ✓


## 8. Memory Comparison - Dense vs Sparse

The key advantage of sparse representation:
- **Dense**: 32K × 32K × 4 bytes = **4 GB per HRT**
- **Sparse**: 100K edges × 20 bytes = **~2 MB**
- **Savings**: ~2000x smaller!

In [8]:
print("=== Memory Comparison ===")
print()

# Dense calculation
dim = config.dimension
dense_bytes = dim * dim * 4  # float32
dense_gb = dense_bytes / (1024**3)
print(f"Dense AM ({dim:,} × {dim:,} float32):")
print(f"  {dense_gb:.1f} GB per HRT")
print(f"  {dense_gb * 3:.1f} GB for 3 HRTs (would OOM on 64GB laptop!)")
print()

# Sparse calculation (for 100K edges)
n_edges = 100_000
sparse_bytes = n_edges * 20  # 2 int64 + 1 float32
sparse_mb = sparse_bytes / (1024**2)
print(f"Sparse AM ({n_edges:,} edges):")
print(f"  {sparse_mb:.1f} MB per HRT")
print(f"  {sparse_mb * 3:.1f} MB for 3 HRTs (easily fits in GPU!)")
print()

# Ratio
ratio = dense_bytes / sparse_bytes
print(f"Memory savings: {ratio:.0f}x smaller!")
print()

# Actual HRT memory
print(f"Current HRT ({hrt3.nnz} edges): {hrt3.memory_mb():.4f} MB")

=== Memory Comparison ===

Dense AM (32,770 × 32,770 float32):
  4.0 GB per HRT
  12.0 GB for 3 HRTs (would OOM on 64GB laptop!)

Sparse AM (100,000 edges):
  1.9 MB per HRT
  5.7 MB for 3 HRTs (easily fits in GPU!)

Memory savings: 2148x smaller!

Current HRT (3 edges): 0.0004 MB


## 9. Large Scale Test - 100K Edges on GPU

Creating a large sparse HRT with 100K edges:

In [9]:
print("=== Large Scale Test ===")
print()

# Generate 100K edges
n_edges = 100_000
edges = [
    (i % config.dimension, (i + 1) % config.dimension, float(i % 100 + 1))
    for i in range(n_edges)
]
print(f"Generated {len(edges):,} edges")

# Create via batch (fast path)
start = time.time()
large_am = SparseAM.from_edges(config, edges)
am_time = time.time() - start
print(f"SparseAM created in {am_time*1000:.1f}ms")

# Build lattice
start = time.time()
large_lattice = SparseLattice.from_sparse_am(large_am)
lattice_time = time.time() - start
print(f"SparseLattice built in {lattice_time*1000:.1f}ms")

# Complete HRT
start = time.time()
large_hrt = SparseHRT(
    am=large_am,
    lattice=large_lattice,
    config=config
)
hrt_time = time.time() - start
print(f"SparseHRT assembled in {hrt_time*1000:.1f}ms")
print()

print(f"=== Result ===")
print(f"{large_hrt}")
print(f"Active rows: {len(large_hrt.am.active_rows):,}")
print(f"Active cols: {len(large_hrt.am.active_cols):,}")
print()

# Query performance
start = time.time()
for _ in range(1000):
    _ = large_hrt.am.get(50000, 50001)
query_time = (time.time() - start) / 1000 * 1_000_000
print(f"Query time: {query_time:.1f}μs per lookup")

=== Large Scale Test ===

Generated 100,000 edges
SparseAM created in 67.2ms
SparseLattice built in 269.0ms
SparseHRT assembled in 0.1ms

=== Result ===
SparseHRT(nnz=100000, lut=0, step=0, mem=5.91MB, device='cuda:0')
Active rows: 32,770
Active cols: 32,770

Query time: 110.1μs per lookup


## 10. Merge Performance

Testing merge of two large sparse HRTs:

In [10]:
# Create another large HRT with overlapping edges
edges2 = [
    (i % config.dimension, (i + 2) % config.dimension, float(i % 50 + 1))
    for i in range(n_edges)
]
large_am2 = SparseAM.from_edges(config, edges2)
large_lattice2 = SparseLattice.from_sparse_am(large_am2)
large_hrt2 = SparseHRT(am=large_am2, lattice=large_lattice2, config=config)

print(f"HRT 1: {large_hrt.nnz:,} edges")
print(f"HRT 2: {large_hrt2.nnz:,} edges")
print()

# Merge
start = time.time()
merged_hrt = large_hrt.merge(large_hrt2)
merge_time = time.time() - start

print(f"=== Merge Result ===")
print(f"Merged HRT: {merged_hrt.nnz:,} edges")
print(f"Merge time: {merge_time*1000:.1f}ms")
print(f"Memory: {merged_hrt.memory_mb():.2f} MB")

HRT 1: 100,000 edges
HRT 2: 100,000 edges

=== Merge Result ===
Merged HRT: 65,540 edges
Merge time: 305.5ms
Memory: 5.75 MB


## Summary

| Feature | Dense HRT | Sparse HRT |
|---------|-----------|------------|
| Memory (32K×32K) | 4 GB | ~2 MB (100K edges) |
| 3 HRTs | 12 GB (OOM!) | ~6 MB |
| Device | CPU | CUDA GPU |
| IICA Properties | ✓ | ✓ |
| Merge | O(n²) | O(nnz) |
| Query | O(1) | O(nnz) worst, O(1) avg |

**Version**: Fractal Manifold Core v0.5.0 (Sparse GPU Architecture)

**Key Insight**: AM[i,j] = context covariance, not term frequency!